### Nota importante:

Este notebook utiliza el paquete `spacy`, que no es compatible con JupyterLite. Por tanto, ejecutar el codigo del notebook desde la URL del curso (https://dime-worldbank.github.io/taller-python-RD/lab/index.html) no es posible.

Si deseas ejecutar este contenido, descarga el notebook y ejecutalo desde tu computadora con Jupyter Notebook o Jupyter Lab.

# 2. Limpieza y preparacion de datos de textos

Para la siguiente parte, utilizaremos el archivo "muestra_ejercicio_26_5.xlsx". Este archivo contiene pequenos textos con el detalle de ventas reportadas por empresas en Republica Dominicana. Durante el resto de la sesion, vamos a trabajar con estos textos para modificarlos hasta que esten en una forma que sea util para analisis de datos de texto.

Comenzaremos con explorar los datos para entender que contienen.

### Exploracion de datos de texto

La exploracion de datos de texto no es muy distinta a la exploracion de datos numericos. Basicamente consiste en visualizar un dataframe con datos de texto para familiarizarnos con lo que contiene.

Empezaremos por cargar los datos. Por conveniencia, llamaremos a nuestro datadrame con los datos `df`:

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('datos/muestra_ejercicio_26_5_2025.xlsx', sheet_name='Muestra_clasificada')

In [ ]:
len(df)

In [ ]:
df.head()

Cada fila corresponde a un item reportado en el detalle de una boleta de venta. Para cada item, tenemos:
- El nombre reportado
- El tipo de ingreso por la venta
- Si es un bien o servicio
- La actividad del comercio emisor
- El nombre generico del producto
- El grupo del producto

Vamos a tabular algunas de estas columnas para tener una idea de cuan frecuentes son las categorias que se muestran:

In [ ]:
df['indicador_bien_servicio'].value_counts()

In [ ]:
df['GENERICO'].value_counts()

In [ ]:
df['CLASIFICACION'].value_counts()

In [ ]:
df['nombre_item'].value_counts()

La tabulacion de `nombre_item` indica varios puntos:
- La columna tiene muchas categorias unicas, que no se repiten
- La mayoria de caracteres estan en mayusculas
- La mayoria de categorias describen en una o dos palabras el producto
- Algunas tambien tienen informacion adicional en numeros o codigos

Ahora que ya conocemos mejor los datos, podemos comenzar a planear que hacer con ellos. Para esto, vamos a suponer que tenemos dos objetivos para estos datos:

- Hacer un analisis descriptivo de los textos en `nombre_item`.
    - Contar palabras
    - Hacer una nube de palabras
- Contruir un clasificador automatico que toma los textos en `nombre_item` y determina si el item es de la categoria alimento o no.

En concreto, realizaremos esto en la sesion de manana. Antes de eso, tenemos que preparar nuestros datos para que esten en una forma que resulta la mas adecuada para estos tipos de analisis.

## 2.1 Normalizacion de datos de texto

Para comprender por que la preparacion de textos es necesario, pensemos sobre lo siguiente?

- Nuestros textos están en un estado muy "crudo". ¿No deberíamos "limpiarlos" un poco antes de contar palabras? Por ejemplo, palabras en minusculas y mayusculas se cuentan por separado  
- Los textos en espanol (como en muchos otros idiomas) suelen repetir muchas palabras que no aportan mucho son muy informativas, como preposiciones, pronombres o contracciones. ¿Podemos eliminar algunas de ellas antes de contar palabras?  
- Por último, ¿no deberíamos contar en la misma categoría palabras que no son exactamente iguales pero tienen un significado muy similar? Por ejemplo:
    + diferentes conjugaciones del mismo verbo
    + formas singulares y plurales del mismo sustantivo

La respuesta a todas estas preguntas es **sí**. Lo haremos en la preparacion de los datos. La preparacion de datos en análisis de texto es **extremadamente importante**. Omitirlo puede darte resultados muy diferentes en tareas de análisis de texto.

El preprocesamiento y la preparacion puede incluir múltiples tareas. Aplicaremos las siguientes a nuestros textos:

- Convertir caracteres a minúsculas  
- Tokenización: transformar textos en listas de palabras  
- Eliminar palabras poco informativas. En lenguaje tecnico se conocen como *stop words*
- Lematización: transformar diferentes formas de palabras en una forma común que exprese un significado similar. Esto es útil para "normalizar" conjugaciones de verbos o formas 

Por suerte, existe un paquete de Python muy útil que podemos usar para esto: [spaCy](https://spacy.io/).  
spaCy pone a nuestra disposición modelos de NLP (procesamiento de lenguaje natural) que permiten tokenizar, lematizar y detectar *stop words* y caracteres que no son palabras (como dígitos o signos de puntuación), por lo que podemos transformar fácilmente un texto en una lista de palabras lematizadas "significativas" que podemos usar para contar palabras o construir clasificadores.

### Trabajando con spaCy

Primero necesitamos cargar spaCy.

In [ ]:
# Instala spaCy si no lo tienes:
#!pip install spacy

In [ ]:
import spacy

Ahora necesitamos **descargar** el modelo de NLP de spaCy en espanol. spaCy tiene varios modelos en espanol. Puedes verlos [aqui](https://spacy.io/models/es%20). Los modelos de mayor tamano funcionan mejor, pero tambien requieren mas espacio de disco duro y poder computacional. Nosotros trabajaremos con el modelo `es_core_news_md`, que no es muy pesado y logra buenos resultados.

In [ ]:
#!python -m spacy download es_core_news_md

Ahora **cargamos** el modelo para que esté disponible en este notebook:

In [ ]:
nlp = spacy.load('es_core_news_md')

Luego, construiremos una función que:

1. Lee un texto  
1. Lo transforme a minúsculas  
1. Lo cargue al modelo de spaCy
1. Para cada palabra de mas de 2 caracteres, obtiene la versión lematizada de aquellas que **no sean**:  
    - *Stop words* (palabras sin mucho significado)
    - Signos de puntuación  
    - Números  
    - Espacios  
1. Finalmente, la función devuelve una lista de las palabras lematizadas

In [ ]:
def normalizacion_tokenizacion(texto):
    
    texto = texto.lower() # minusculas
    doc = nlp(texto)      # cargando el texto al modelo de spaCy

    palabras_normalizadas = []
    for palabra in doc:
        if palabra.text != '\n' \
        and not palabra.is_stop \
        and not palabra.is_punct \
        and not palabra.like_num \
        and len(palabra.text.strip()) > 2:
            lema = str(palabra.lemma_)
            palabras_normalizadas.append(lema)
    
    return palabras_normalizadas

Para entender mejor que es la lema, veamos un ejemplo:

In [ ]:
texto = 'dominicanas'
texto = nlp(texto)
for palabra in texto:
    print(palabra.lemma_)

Para entender mejor lo que hace la función, veamos el resultado para un par de textos:

In [ ]:
texto = df['nombre_item'][2]
print(texto)

In [ ]:
normalizacion_tokenizacion(texto)

In [ ]:
texto = df['nombre_item'][263]
print(texto)

In [ ]:
normalizacion_tokenizacion(texto)

Notemos lo siguiente:

- El resultado es una lista, no un texto
- todas las palabras estan en minusculas
- Las preposiciones han sido eliminadas
- Los plurales han sido transformados a singular como parte de la lematizacion
- Algunos femeninos han sido transformados a masculino tambien como parte de la lematizacion

Para nuestra sorpresa, tambien podemos ver que la palabra `3x5` no ha sido eliminada. Esto lo corregiremos posteriormente mediante el uso de expresiones regulares.

Por ahora, lo que nos queda es aplicar la funcion `normalizacion_tokenizacion()` de forma transversal a toda la columna `nombre_item`.

In [ ]:
df['nombre_item_norm'] = df['nombre_item'].apply(normalizacion_tokenizacion)

In [ ]:
df

In [ ]:
df.head(10)

Visualizar estos resultados nos hace dar cuenta que hay mas codigos que no han sido eliminados en la normalizacion:

- `3x5`
- `rf-88`
- `ib1707`

Corregiremos esto mediante expresiones regulares.



## 2.2 Patrones en textos y expresiones regulares

Estos tres codigos parecen seguir patrones en los caracteres que contienen:

- `3x5`: un numero, seguido de una `x`, seguida de un numero
- `rf-88`: dos letras, seguidas de un guion, seguidas de dos numeros
- `ib1707`: dos letras, seguidas de cuatros numeros

Vamos a aprovechar estos patrones para eliminarlos del texto normalizado. Usaremos expresiones regulares para esto.

### Expresiones regulares

En programación, las expresiones regulares (*regex*) son secuencias de caracteres que coinciden con un patrón dentro de un texto. Un ejemplo simple:

In [ ]:
# re es el paquete en Python para trabajar con expresiones regulares.
import re

In [ ]:
texto = 'El número de DNI del contribuyente 1 es 30551. Nació el 1 de julio de 1976. El contribuyente 2 cuenta con DNI 71098'

# Patrón para capturar los ID en este texto: secuencias de cinco caracteres numéricos.
patron = '\d{5}'

# Capturando los DNI
dni = re.findall(patron, texto)
print(dni)

- `\d` es un codigo representa un número (0-9). Esto es equivalente a `[0-9]`
- `{5}` significa que el carácter anterior en el patrón se repite cinco veces
- Una variación de este patrón podría ser `\d{4}`, que podría usarse para capturar años. Esto habría devuelto una lista con `1996` en el ejemplo anterior

En regex, existe un codigo para casi todo. Algunos ejemplos:

- Codigos para caracteres:
    + `\d` --> dígitos (0-9)  
    + `\w` --> cualquier caracter de palabra (letras mayúsculas y minúsculas, dígitos numericos y el guion bajo `_`)  
    + `\n` --> saltos de línea  
    + `\s` --> caracteres de espacio en blanco, incluyendo saltos de línea  
    + `.` --> cualquier caracter excepto el salto de línea (\n)  

- Para repetición de caracteres:
    + `{a}` --> el carácter anterior, repetido exactamente "a" veces  
    + `{a,b}` --> el carácter anterior, repetido entre "a" y "b" veces  
    + `*` --> el carácter anterior, repetido cero o más veces  
    + `+` --> el carácter anterior, repetido una o más veces  

Regex puede detectar prácticamente cualquier patrón que podamos imaginar. Sin embargo, trabajar con expresiones regulares puede ser complejo al principio. En esta sesión hemos introducido el concepto de regex para que sepas que existe y que puede usarse para crear columnas en conjuntos de datos que contienen corpus de documentos.

No te preocupes si todavía no entendiste bien cómo funcionan los patrones. Si te interesa aprender más sobre regex, te recomendamos los siguientes recursos:

- Un buen tutorial sobre regex [aquí](https://regexone.com/)  
- Una excelente herramienta visualizadora de expresiones regulares está [aquí](https://jex.im/regulex/#!flags=&re=www%5C.%5Ba-zA-Z0-9-%5D%2B%5C.(%3F%3Acom%7Cnet%7Corg))

### Reemplazando información usando regex

El comando `re.findall()` se usa para extraer todas las menciones de una regex. Nosotros usaremos `re.match()`, que se usa para reemplazar verificar si un patron existe en un texto.

Nuestros patrones seran 3, de acuerdo a lo que ya hemos visto:

- un numero, seguido de una `x`, seguida de un numero
- dos letras minusculas, seguidas de un guion, seguidas de dos numeros
- dos letras minusculas, seguidas de cuatros numeros

In [ ]:
patron1 = '\dx\d'
patron2 = '[a-z]{2}-\d{2}'
patron3 = '[a-z]{2}\d{4}'

Y haremos una funcion que borra estos patrones en `nombre_item_norm`, que era la columna con las palabras normalizadas y en listas. Recuerda que el input de esta funcion debe ser una lista, no un texto:

In [ ]:
def eliminar_codigos(lista_palabras):
    
    nueva_lista = []
    
    for palabra in lista_palabras:
        
        if not re.match(patron1, palabra) and \
            not re.match(patron2, palabra) and \
            not re.match(patron3, palabra):
            
            nueva_lista.append(palabra)
    
    return nueva_lista

Para comprobar su funcionamiento, probaremos la funcion en la observacion de indice 263 de `nombre_item_norm`:

In [ ]:
df['nombre_item_norm'][263]

In [ ]:
eliminar_codigos(df['nombre_item_norm'][263])

Ahora aplicaremos la funcion a toda la columna:

In [ ]:
df['nombre_item_final'] = df['nombre_item_norm'].apply(eliminar_codigos)

In [ ]:
df

Excelente! La columna resultante `nombre_item_final` contiene el texto preparado como lo vamos a necesitar para un analisis descriptivo y para construir un clasificador automatico basico.

# 3. Guardando el resultado

Este resultado contiene dos columnas que contienen valores en listas. Por tanto, no es posible guardar el dataframe como un CSV y preservar toda la informacion que esas listas contienen.

Como alternativa, guardaremos el dataframe como un **pickle**.

## Que es un pickle?

Un pickle es un tipo de archivo que resulta de guardar una **variable** de Python (cualquier tipo de variable realmente) en bytes en nuestra computadora. Puede entenderse el tipo "nativo" de Python para guardar archivos, similar a como son los archivos `.xlsx` de Excel, `.dta` en Stata o `.Rds` para dataframes en R.

## Como crear un pickle?

Normalmente, el codigo para guardar un pickle es este:

```{python}
# Guardando una lista en un pickle
mi_lista = [1, 2, 3, 4]

import pickle
with open('data.pkl', 'wb') as archivo:
    pickle.dump(mi_lista, archivo)
```

Convenientemente, pandas con cuenta con un metodo que podemos aplicar directamente a un dataframe para guardar un pickle: `.to_pickle()`. Este funciona de forma muy similar a `.to_csv()`. Luego podemos usar la funcion de pandas `read_pickle()` para leer el pickle que guardamos.

In [ ]:
ruta_archivo = 'datos/detalle_ventas_clasificadas.pkl'
df.to_pickle(ruta_archivo)

# Comentarios finales

Antes de culminar, repasemos todos lo que hemos visto hoy:

- Lectura de archivos PDF
- Estructuracion de textos no estructurados en dataframes
- Generacion de nuevas columnas sobre atributos del texto
- Normalizacion de textos y tokenizacion
- Expresiones regulares

Todas estas tareas son tareas comunes en analisis de textos para preparar los datos. Manana veremos como utilizar estos datos preparados para realizar un analisis descriptivo y construir un clasificador. La sesion de manana empezara desde el pickle que hemos creado hoy.